In [1]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from sklearn.utils import resample
from IPython.display import display
import numpy as np

# Create a connection with SQL server to get data.
def exec_sql_query(query, param=None):
    
    from sqlalchemy import create_engine
    import urllib
    params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=LAPTOP-C3LFVOFI;DATABASE=student_life;UID=student_sense;PWD=abhinav123")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    connection = engine.raw_connection()
    
    try:
        cursor = connection.cursor()
        if(param):
            cursor.execute(query, param)
        else : 
            cursor.execute(query)
            
        results = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame.from_records(results, columns=columns)
        cursor.close()
        connection.commit()
    finally:
        connection.close()
    
    del engine
    return df

In [2]:

# Maintaining a feature list.
feature_map = {

#     "activity_details" : "SELECT activity_time ,student_id ,activity_inference FROM activity_details where student_id = 1"
    
    "dinning_details" : "SELECT dinning_time ,student_id ,venue_id ,meal_type_id FROM dinning_details where student_id = 1",
    
#     "call_log_details" : "select call_time, student_id, call_duration_min, call_type from call_log_details where student_id = 1",
    
    "sms_details" : "select timestamp, student_id, 1 from sms_details where student_id = 1",
    
    "audio_details" : "select audio_activity_time, student_id, audio_activity_inference from audio_details where student_id = 1",
    
    "conversation_details" : "select conv_start_timestamp, student_id, conv_duration_min from conversation_details where student_id = 1",
    
    "dark_details" : "select dark_start_timestamp, student_id, dark_duration_min from dark_details where student_id = 1",
    
    "phonecharge_details" : "select start_timestamp, student_id, phonecharge_duration_min from phonecharge_details where student_id = 1",

    "phonelock_details" : "select start_timestamp, student_id, phonelock_duration_min from phonelock_details where student_id = 1",
     
    "gps_details" : "select wifi_timestamp as time, student_id, latitude, longitude from gps_details where student_id = 1"
    
                         }

# Getting Raw Data.

data_dict = {}

# Processing Stress Levels.
stress_details_raw = exec_sql_query("select student_id, response_time, stress_level from stress_details where student_id = 1")
stress_details = stress_details_raw.loc[:,["response_time","student_id","stress_level"]]
stress_details = stress_details.sort_values(by="response_time")
stress_details.rename({"response_time": "time"}, axis='columns', inplace=True)

for key in query_map.keys():
    
    feature_query = query_map[key]
    
    # Data processing begins..
    feature_data = exec_sql_query(feature_query)
    
    
    # Selecting Time Col.
    train_col_list = []
    
    for col in feature_data.columns:
        if "time" in col:
            time_column = col
        else:
            train_col_list.append(col)
    
    feature_data.rename({time_column: "time"}, axis='columns', inplace=True)
    
    time_column = "time"
    # Sorting by values of time.        
    feature_data = feature_data.sort_values(by=time_column)
    feature_data = feature_data.dropna()
    train_feature_data = pd.merge_asof(stress_details,
                                 feature_data,
                                 direction="nearest",
                                 by="student_id",
                                 on = "time",
                                 tolerance=pd.Timedelta("1 hour"))
    
    # Creating New column for activity_details
    feature_data["stress_level"] = np.nan 
    
    # Sorting column naming convention
    train_col_list = []
    
    for col in train_feature_data.columns:
        if col !="stress_level":
            train_col_list.append(col)
    
    train_col_list.append('stress_level')
    train_feature_data = train_feature_data[train_col_list]
    train_feature_data = train_feature_data.append(feature_data, ignore_index=True)
    train_feature_data.sort_values(by=["student_id","time"], inplace=True)
    train_feature_data.drop_duplicates()
    train_feature_data.reset_index(inplace=True)
    train_feature_x = train_feature_data.iloc[:,1:-1]
    train_feature_x = train_feature_x.fillna(method="backfill", axis=0)
    train_feature_y = train_feature_data.loc[:,["stress_level"]]
    df_filter = train_feature_y.stress_level.notnull()
    train_feature_y_indices = train_feature_y[df_filter].index
    train_feature_y_indices = pd.DataFrame(train_feature_y_indices)
    train_feature_y_indices.rename({0:"indices"}, axis=1, inplace=True)
#     print(type(train_feature_y_indices))
#     display(train_feature_y_indices)
#     display(train_feature_x.head())
#     display(train_feature_y.head())
#     display(train_feature_data.head())    
    
    
    train_feature_x.to_csv("data/"+key+"_train_x.csv", index=False)
    train_feature_y.to_csv("data/"+key+"_train_y.csv", index=False)    
    train_feature_y_indices.to_csv("data/"+key+"_train_y_indices.csv", index=False)    
    